## VOC에 대한 요약 생성

- 프롬프트 엔지니어링 기반 요약 생성

#### [중요!] 사전에 Bedrock KB (Knowledge Base) 설정은 완료 되어 있다고 가정 합니다.

In [1]:
# 환경 설정
!pip install boto3 -U
!pip install pandas -U
!pip install openpyxl -U

### Amazon Bedrock Client 설정

In [5]:
import boto3
import pprint
from botocore.client import Config
import json

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_runtime = boto3.client('bedrock-runtime', region_name = "us-east-1") ## Claude 리전


### 요약 함수 정의

In [8]:
def summarize_text(text):
    prompt = f"""
    다음 텍스트의 핵심 내용을 정확하고 간결하게 요약해주세요: 
    
    <text>
    {text}
    </text>
    
    요약 지침:
    1. 반드시 30자 이내로 작성하세요.
    2. 가장 중요한 정보만 포함하세요.
    3. 완전한 문장으로 작성하지 말고, 핵심 키워드나 구문만 사용하세요.
    4. 불필요한 조사, 접속사는 생략하세요.
    5. 원문의 주요 의도나 감정을 반영하세요.
    
    응답 형식은 다음 예시 <example></example>를 참조하세요.
    <example>
    자녀가 만원을 내고 5000원짜리 구슬 아이스크림을 사먹었는데 거스름돈을 돌려주지 않았음. 아이가 잔돈을 달라고 했으나 근무자가 대꾸도 안하고 무시했다는 이야기를 들어 활당했으니 개선 바람.
    </example>
    
    응답 형식: 50자 이내로 서술형 문장 1개 또는 2개 이내로 요약된 내용을 표현하세요. 일부 추가 설명은 포함되어도 됩니다. 생성된 답변만 출력하세요.
    """

    response = bedrock_runtime.converse(
        modelId="us.anthropic.claude-3-5-sonnet-20240620-v1:0",   ## Claude 3.5 Sonnet
        #modelId="us.anthropic.claude-3-haiku-20240307-v1:0",    ## Claude 3 Haiku
        messages=[
            {
                "role": "user",
                "content": [{"text": prompt}]
            }
        ],
        inferenceConfig={
            "maxTokens": 4096,
            "temperature": 0.7,
            "topP": 1
        }
    )
    
    # 응답 구조에 따라 텍스트 추출
    if 'output' in response and 'message' in response['output']:
        return response['output']['message']['content'][0]['text']
    else:
        # 응답 구조가 예상과 다를 경우 전체 응답 반환
        return str(response)

### 사용자 쿼리 (입력 VOC)

In [10]:
# 입력하는 VOC 
query = """
안녕하세요. 동물원 체험 프로그램에 대해 문의드립니다. 지난번 공지사항에 매주 토요일마다 동물 먹이주기 체험을 진행한다고 하셨는데, 모든 동물에게 해당되는 건가요? 기린이랑 코끼리만 먹이주기 체험을 하고 다른 동물들은 제외된다면 좀 아쉬울 것 같아요. 공지사항만 봐서는 모든 동물이 포함되는 것처럼 보였는데, 혹시 잘못 이해한 건 아닌지 확인 차 여쭤봅니다.
모든 동물들에게 골고루 관심을 가져주셨으면 좋겠어요. 요즘 SNS에서 기린과 코끼리 위주로 홍보하시는 걸 보고 다른 동물 팬들이 많이 아쉬워하더라고요. 저도 그동안 다른 동물 팬들의 요청으로 민원을 자주 넣었지만, 이제는 그냥 구경만 하고 있어요.
하지만 계속 특정 동물들만 홍보하신다면 팬들이 계속해서 불만을 제기할 것 같아요. 그래도 지난번 '펭귄의 날' 행사는 정말 좋았습니다. 앞으로도 다양한 동물들을 위한 행사 많이 해주세요. 늘 동물원 소식 전해주셔서 감사합니다. 
"""

# 기존 summarize_text() 함수를 활용하여 쿼리 처리
summary = summarize_text(query)
print("=== 쿼리 요약 ===\n")
print(summary)

=== 쿼리 요약 ===

동물 먹이주기 체험 범위 문의, 다양한 동물 홍보 요청, 펭귄 행사 호평
